Import duckdb to be able to use sql

In [60]:
import duckdb
import pandas as pd
import re

with duckdb.connect("data/sakila.duckdb") as conn:
    description = conn.sql("DESC;").df()

description

,database,schema,name,column_names,column_types,temporary
0,sakila,staging,_dlt_loads,"[load_id, schema_name, status, inserted_at, sc...","[VARCHAR, VARCHAR, BIGINT, TIMESTAMP WITH TIME...",False
1,sakila,staging,_dlt_pipeline_state,"[version, engine_version, pipeline_name, state...","[BIGINT, BIGINT, VARCHAR, VARCHAR, TIMESTAMP W...",False
2,sakila,staging,_dlt_version,"[version, engine_version, inserted_at, schema_...","[BIGINT, BIGINT, TIMESTAMP WITH TIME ZONE, VAR...",False
3,sakila,staging,actor,"[actor_id, first_name, last_name, last_update,...","[DECIMAL(38,9), VARCHAR, VARCHAR, TIMESTAMP, V...",False
4,sakila,staging,address,"[address_id, address, address2, district, city...","[BIGINT, VARCHAR, VARCHAR, VARCHAR, BIGINT, VA...",False
5,sakila,staging,category,"[category_id, name, last_update, _dlt_load_id,...","[BIGINT, VARCHAR, TIMESTAMP, VARCHAR, VARCHAR]",False
6,sakila,staging,city,"[city_id, city, country_id, last_update, _dlt_...","[BIGINT, VARCHAR, BIGINT, TIMESTAMP, VARCHAR, ...",False
7,sakila,staging,country,"[country_id, country, last_update, _dlt_load_i...","[BIGINT, VARCHAR, TIMESTAMP, VARCHAR, VARCHAR]",False
8,sakila,staging,customer,"[customer_id, store_id, first_name, last_name,...","[BIGINT, BIGINT, VARCHAR, VARCHAR, VARCHAR, BI...",False
9,sakila,staging,film,"[film_id, title, description, release_year, la...","[BIGINT, VARCHAR, VARCHAR, VARCHAR, BIGINT, BI...",False


storing data into dictionary with dfs

In [ ]:
dfs = {}

with duckdb.connect("data/sakila.duckdb") as conn:
    for name in description["name"]:
        if name[0] != "_":
            dfs[name] = conn.sql(f"FROM staging.{name};").df()

            duckdb.register(name, dfs[name])

dfs[""].head()


,actor_id,film_id,last_update,_dlt_load_id,_dlt_id
0,1,1,2021-03-06 15:52:45,1764933302.114754,1qozovIAdq5C7Q
1,1,23,2021-03-06 15:52:45,1764933302.114754,rrKerqh1osw1Qg
2,1,25,2021-03-06 15:52:45,1764933302.114754,JQJj0YBtPoszeA
3,1,106,2021-03-06 15:52:45,1764933302.114754,RTID1/Vhn82y6Q
4,1,140,2021-03-06 15:52:45,1764933302.114754,d0W0TjNENXT2Pw


Task 1

a)

In [62]:
duckdb.sql("""
    SELECT 
           title, length 
    FROM film
    WHERE
           length >= 180;
""").df()

,title,length
0,ALLEY EVOLUTION,180
1,ANALYZE HOOSIERS,181
2,BAKED CLEOPATRA,182
3,CATCH AMISTAD,183
4,CHICAGO NORTH,185
5,CONFIDENTIAL INTERVIEW,180
6,CONSPIRACY SPIRIT,184
7,CONTROL ANTHEM,185
8,CRYSTAL BREAKING,184
9,DARN FORRESTER,185


b)

In [151]:
duckdb.sql("""
    SELECT title, 
           rating, 
           length, 
           description
    FROM film
    WHERE (' ' || title || ' ') LIKE '% LOVE %'
    --WHERE title ~ '(^|[^A-Za-z0-9_])love($|[^A-Za-z0-9_])'
    --WHERE title ~ '(?i)(^| )love( |$)'
    --WHERE title LIKE 'LOVE %'
    --    OR title LIKE '% LOVE %' 
    --    OR title LIKE 'LOVE'
    --    OR title LIKE '% LOVE'; 
    ;
""").df()

,title,rating,length,description
0,GRAFFITI LOVE,PG,117,A Unbelieveable Epistle of a Sumo Wrestler And...
1,IDAHO LOVE,PG-13,172,A Fast-Paced Drama of a Student And a Crocodil...
2,INDIAN LOVE,NC-17,135,A Insightful Saga of a Mad Scientist And a Mad...
3,LAWRENCE LOVE,NC-17,175,A Fanciful Yarn of a Database Administrator An...
4,LOVE SUICIDES,R,181,A Brilliant Panorama of a Hunter And a Explore...


In [ ]:
#using pandas for b
filtered_df = dfs["film"][dfs["film"]['title'].str.contains(r"\blove\b", case=False, na=False)]
result = filtered_df[['title', 'rating', 'length', 'description']]

print(result)


             title rating  length  \
373  GRAFFITI LOVE     PG     117   
447     IDAHO LOVE  PG-13     172   
457    INDIAN LOVE  NC-17     135   
510  LAWRENCE LOVE  NC-17     175   
534  LOVE SUICIDES      R     181   

                                           description  
373  A Unbelieveable Epistle of a Sumo Wrestler And...  
447  A Fast-Paced Drama of a Student And a Crocodil...  
457  A Insightful Saga of a Mad Scientist And a Mad...  
510  A Fanciful Yarn of a Database Administrator An...  
534  A Brilliant Panorama of a Hunter And a Explore...  


c)

In [ ]:
duckdb.sql("""
    SELECT 
            
    FROM film
""").df()

BinderException: Binder Error: column "title" must appear in the GROUP BY clause or must be part of an aggregate function.
Either add it to the GROUP BY list, or use "ANY_VALUE(title)" if the exact value of "title" is not important.